In [23]:
import pandas as pd
import numpy as np
import requests
import copy

In [24]:
LabGene = pd.read_excel("LabGene.xlsx", sheet_name="LabGene")
LabGene.head()

,Locus tag,ID,Gene symbol,Modules,备注
0,Rv0001,885041,dnaA,NaN,NaN
1,Rv0002,887092,dnaN,NaN,NaN
2,Rv0005,887081,gyrB,NaN,NaN
3,Rv0006,887105,gyrA,NaN,NaN
4,Rv0007,885982,Rv0007,NaN,NaN


In [25]:
LabGene.set_index(["Locus tag"],inplace=True)
LabGene.head()

,ID,Gene symbol,Modules,备注
Locus tag,,,,
Rv0001,885041,dnaA,NaN,NaN
Rv0002,887092,dnaN,NaN,NaN
Rv0005,887081,gyrB,NaN,NaN
Rv0006,887105,gyrA,NaN,NaN
Rv0007,885982,Rv0007,NaN,NaN


In [26]:
ModuleGene = pd.read_excel("ModuleGene.xlsx", sheet_name="ModuleGene")
ModuleGene.head()

,using Gene
Module 1,NaN
Module 2,NaN
Module 3,NaN
Module 4,NaN
Module 5,NaN


In [27]:
def parseAGene(GeneTextTd):
    text = GeneTextTd[GeneTextTd.find("<a href"):GeneTextTd.find("</a>") + 1]
    text = text[text.find(">") + 1: -1]
    return text

In [82]:
def parseAModule(moduleNum):
    try:
        response = requests.request("get", "http://networks.systemsbiology.net/mtb/biclusters/bicluster_0" + moduleNum, timeout = 20)
    except:
        print("error")
        return []
    finally:
        print("Module", moduleNum)
    
    if(response.status_code == 200):
        text = str(response.text.encode("utf-8"))
        text = text[text.find("views-view-grid cols-2") + 10 : -1]
        text = text[0: text.find("</table>")]
        
        genes = []
        tdBegin = text.find("<td >")
        while(tdBegin != -1):
            tdEnd = text.find("</td>")
            GeneTextTd = text[tdBegin:tdEnd]
            text = text[tdEnd + 4 : -1]
            gene = parseAGene(GeneTextTd)
            if(gene != ""):
                genes.append(gene)
            tdBegin = text.find("<td >")
        return genes
    else:
        return []
    

In [29]:
def formalNum(num):
    strNum = str(num)
    strLen = len(strNum)
    if(strLen == 3):
        return strNum
    elif(strLen == 2):
        return "0" + strNum
    else:
        return "00" + strNum

In [30]:
def main(low, high):
    for moduleNum in range(low, high + 1):
        genes = parseAModule(formalNum(moduleNum))
        print(genes)
        if(genes == []):
            ModuleGene.loc["Module " + str(moduleNum)] = "skip"
            continue
        for gene in genes:
            if((LabGene.index == gene).any()):
                if(not LabGene.loc[gene, "Modules"] == LabGene.loc[gene, "Modules"]):
                    LabGene.loc[gene, "Modules"] = "Module " + str(moduleNum)
                else:
                    LabGene.loc[gene, "Modules"] = LabGene.loc[gene, "Modules"] + "%" + "Module " + str(moduleNum)
        ModuleGene.loc["Module " + str(moduleNum)] = "%".join(genes)

In [31]:
main(1, 50)#此处手动分开爬是因为内存会炸。。。。。。。

Module 000
[]
Module 001
['Rv0830', 'Rv2377c', 'Rv2963', 'Rv2975c', 'Rv3848']
Module 002
['Rv0472c', 'Rv1871c', 'Rv3005c', 'Rv3006', 'Rv3037c', 'Rv3781', 'Rv3783', 'Rv3805c']
Module 003
['Rv0395', 'Rv0396', 'Rv0689c', 'Rv0793', 'Rv0843', 'Rv0915c', 'Rv1137c', 'Rv1414', 'Rv1526c', 'Rv1705c', 'Rv1706c', 'Rv1729c', 'Rv1739c', 'Rv1819c', 'Rv2307c', 'Rv2311', 'Rv2312', 'Rv2407', 'Rv3121', 'Rv3533c']
Module 004
['Rv1408', 'Rv1409', 'Rv1923', 'Rv2305', 'Rv3403c', 'Rv3589', 'Rv3768']
Module 005
['Rv0809', 'Rv1101c', 'Rv1192', 'Rv1193', 'Rv1627c', 'Rv1628c', 'Rv1629', 'Rv2749', 'Rv2750']
Module 006
['Rv0345', 'Rv0509', 'Rv0510', 'Rv0513', 'Rv1089', 'Rv1173', 'Rv1207', 'Rv1208', 'Rv2425c', 'Rv2477c', 'Rv2726c', 'Rv2995c', 'Rv3025c', 'Rv3103c', 'Rv3104c', 'Rv3480c', 'Rv3803c', 'Rv3843c']
Module 007
['Rv0212c', 'Rv0213c', 'Rv0214', 'Rv0595c', 'Rv0596c', 'Rv0653c', 'Rv0654', 'Rv1364c', 'Rv1985c', 'Rv1986', 'Rv2516c', 'Rv2517c', 'Rv2686c', 'Rv2687c', 'Rv2688c', 'Rv3237c', 'Rv3617', 'Rv3618']
Module 

In [41]:
ModuleGene[0:50]
#error Module 40

,using Gene
Module 1,Rv0830%Rv2377c%Rv2963%Rv2975c%Rv3848
Module 2,Rv0472c%Rv1871c%Rv3005c%Rv3006%Rv3037c%Rv3781%...
Module 3,Rv0395%Rv0396%Rv0689c%Rv0793%Rv0843%Rv0915c%Rv...
Module 4,Rv1408%Rv1409%Rv1923%Rv2305%Rv3403c%Rv3589%Rv3768
Module 5,Rv0809%Rv1101c%Rv1192%Rv1193%Rv1627c%Rv1628c%R...
Module 6,Rv0345%Rv0509%Rv0510%Rv0513%Rv1089%Rv1173%Rv12...
Module 7,Rv0212c%Rv0213c%Rv0214%Rv0595c%Rv0596c%Rv0653c...
Module 8,Rv0656c%Rv1450c%Rv1452c%Rv1453%Rv2079%Rv2352c%...
Module 9,Rv0215c%Rv0236c%Rv1289%Rv2438c%Rv2528c%Rv2590%...
Module 10,Rv1340%Rv1448c%Rv2722%Rv3586


In [93]:
main(40, 40)

Module 040
['Rv2136c', 'Rv3522', 'Rv3638', 'Rv3640c', 'Rv3736']


KeyError: 39

In [36]:
main(51, 100)

Module 051
['Rv0065', 'Rv0323c', 'Rv0326', 'Rv0549c', 'Rv0550c', 'Rv0784', 'Rv0785', 'Rv1674c', 'Rv3415c']
Module 052
['Rv0948c', 'Rv0949', 'Rv2329c', 'Rv2690c', 'Rv2754c', 'Rv3436c', 'Rv0125']
Module 053
['Rv0699', 'Rv0965c', 'Rv0967', 'Rv0968', 'Rv0969', 'Rv0970', 'Rv2489c', 'Rv2671', 'Rv2673', 'Rv2746c', 'Rv2747', 'Rv2766c', 'Rv3366', 'Rv2644c']
Module 054
['Rv1691', 'Rv1692', 'Rv2264c', 'Rv2274c', 'Rv2275', 'Rv2276', 'Rv2861c', 'Rv3279c', 'Rv3280', 'Rv3281', 'Rv3282']
Module 055
['Rv0566c', 'Rv0567', 'Rv0759c', 'Rv0831c', 'Rv1018c', 'Rv1019', 'Rv1020', 'Rv1021', 'Rv1292', 'Rv1293', 'Rv1295', 'Rv1746', 'Rv2272', 'Rv2273', 'Rv2289', 'Rv2290']
Module 056
['Rv0058', 'Rv0361', 'Rv1821', 'Rv2453c', 'Rv2454c', 'Rv2455c', 'Rv2734', 'Rv3035', 'Rv3198c', 'Rv3913', 'Rv3914', 'Rv2732c', 'Rv2733c', 'Rv2855']
Module 057
['Rv0634c', 'Rv0953c', 'Rv0954', 'Rv1065', 'Rv1066', 'Rv1631', 'Rv2070c', 'Rv2071c', 'Rv2072c', 'Rv2073c', 'Rv2209', 'Rv2520c', 'Rv2521']
Module 058
[]
Module 059
['Rv0610c', 'Rv

In [42]:
ModuleGene[50:100]
#error Module 58

,using Gene
Module 51,Rv0065%Rv0323c%Rv0326%Rv0549c%Rv0550c%Rv0784%R...
Module 52,Rv0948c%Rv0949%Rv2329c%Rv2690c%Rv2754c%Rv3436c...
Module 53,Rv0699%Rv0965c%Rv0967%Rv0968%Rv0969%Rv0970%Rv2...
Module 54,Rv1691%Rv1692%Rv2264c%Rv2274c%Rv2275%Rv2276%Rv...
Module 55,Rv0566c%Rv0567%Rv0759c%Rv0831c%Rv1018c%Rv1019%...
Module 56,Rv0058%Rv0361%Rv1821%Rv2453c%Rv2454c%Rv2455c%R...
Module 57,Rv0634c%Rv0953c%Rv0954%Rv1065%Rv1066%Rv1631%Rv...
Module 58,skip
Module 59,Rv0610c%Rv0661c%Rv0662c%Rv1061%Rv1062%Rv1730c%...
Module 60,Rv0391%Rv0608%Rv0238


In [116]:
main(58, 58)

Module 058
['Rv2161c', 'Rv2286c', 'Rv2530c', 'Rv2801c', 'Rv3520c', 'Rv3521', 'Rv3523', 'Rv3405c', 'Rv3522']


In [38]:
main(101, 150)

Module 101
['Rv0143c', 'Rv0144', 'Rv0464c', 'Rv0465c', 'Rv0466', 'Rv1013', 'Rv1274', 'Rv1838c', 'Rv1839c', 'Rv2376c', 'Rv3346c', 'Rv1772']
Module 102
['Rv1056', 'Rv1444c', 'Rv1624c', 'Rv1625c', 'Rv1626', 'Rv2336', 'Rv2459', 'Rv2739c', 'Rv2740', 'Rv2985']
Module 103
['Rv1418', 'Rv2088', 'Rv3143', 'Rv3771c']
Module 104
['Rv0338c', 'Rv0829', 'Rv1118c', 'Rv1381', 'Rv1382', 'Rv1383', 'Rv1384', 'Rv1385', 'Rv1649', 'Rv1650', 'Rv2394', 'Rv2395', 'Rv2402', 'Rv2735c', 'Rv2736c', 'Rv3685c', 'Rv3687c', 'Rv1678', 'Rv1679']
Module 105
['Rv0050', 'Rv0051', 'Rv0380c', 'Rv0486', 'Rv0562', 'Rv1298', 'Rv1299', 'Rv1300', 'Rv1301', 'Rv1333', 'Rv1334', 'Rv1335', 'Rv1336', 'Rv1337', 'Rv1699', 'Rv2868c']
Module 106
['Rv0140', 'Rv1914c', 'Rv1915', 'Rv1916', 'Rv2466c', 'Rv2557', 'Rv2658c', 'Rv2659c', 'Rv2662', 'Rv2664', 'Rv3206c']
Module 107
['Rv0196', 'Rv0197', 'Rv0246', 'Rv0991c', 'Rv2249c', 'Rv2250c', 'Rv2251', 'Rv2252', 'Rv2293c', 'Rv2999', 'Rv3831', 'Rv3839', 'Rv2998', 'Rv3840']
Module 108
['Rv1498c', 'Rv2

In [46]:
ModuleGene[100:150]

,using Gene
Module 101,Rv0143c%Rv0144%Rv0464c%Rv0465c%Rv0466%Rv1013%R...
Module 102,Rv1056%Rv1444c%Rv1624c%Rv1625c%Rv1626%Rv2336%R...
Module 103,Rv1418%Rv2088%Rv3143%Rv3771c
Module 104,Rv0338c%Rv0829%Rv1118c%Rv1381%Rv1382%Rv1383%Rv...
Module 105,Rv0050%Rv0051%Rv0380c%Rv0486%Rv0562%Rv1298%Rv1...
Module 106,Rv0140%Rv1914c%Rv1915%Rv1916%Rv2466c%Rv2557%Rv...
Module 107,Rv0196%Rv0197%Rv0246%Rv0991c%Rv2249c%Rv2250c%R...
Module 108,Rv1498c%Rv2078%Rv2807%Rv2894c
Module 109,Rv1249c%Rv1250%Rv2434c%Rv3125c%Rv3636%Rv0917%R...
Module 110,Rv1789%Rv2115c%Rv2116%Rv2117%Rv2130c%Rv3238c


In [47]:
main(151, 200)

Module 151
['Rv1189', 'Rv2451', 'Rv2543', 'Rv2545', 'Rv3902c', 'Rv3903c', 'Rv2341', 'Rv2489c', 'Rv2662', 'Rv3904c']
Module 152
['Rv0764c', 'Rv0767c', 'Rv1455', 'Rv3063', 'Rv3071', 'Rv3335c', 'Rv3548c', 'Rv3549c', 'Rv3550', 'Rv3551', 'Rv3553', 'Rv3559c', 'Rv3561', 'Rv3564', 'Rv3552', 'Rv3565']
Module 153
['Rv0043c', 'Rv1467c', 'Rv2033c', 'Rv2883c', 'Rv2884', 'Rv0755c', 'Rv0892', 'Rv1511', 'Rv1512', 'Rv1513', 'Rv2670c', 'Rv3179']
Module 154
['Rv0007', 'Rv0147', 'Rv0148', 'Rv0149', 'Rv0821c', 'Rv0822c', 'Rv1162', 'Rv1163', 'Rv1165', 'Rv1689', 'Rv3588c', 'Rv0530', 'Rv1467c', 'Rv2402']
Module 155
['Rv0039c', 'Rv1413', 'Rv2149c', 'Rv2150c', 'Rv2606c', 'Rv2637', 'Rv2923c']
Module 156
['Rv0307c', 'Rv0597c', 'Rv0811c', 'Rv1084', 'Rv1239c', 'Rv1861', 'Rv1862', 'Rv2281', 'Rv3139', 'Rv3140', 'Rv2115c', 'Rv2116', 'Rv2117']
Module 157
['Rv0040c', 'Rv0041', 'Rv0309', 'Rv0690c', 'Rv0692', 'Rv0693', 'Rv0694', 'Rv0888', 'Rv1269c', 'Rv1270c', 'Rv1904', 'Rv2080', 'Rv2822c', 'Rv3763', 'Rv0039c', 'Rv0312', 

In [68]:
ModuleGene[150:200]
#error Module 176 200

,using Gene
Module 151,Rv1189%Rv2451%Rv2543%Rv2545%Rv3902c%Rv3903c%Rv...
Module 152,Rv0764c%Rv0767c%Rv1455%Rv3063%Rv3071%Rv3335c%R...
Module 153,Rv0043c%Rv1467c%Rv2033c%Rv2883c%Rv2884%Rv0755c...
Module 154,Rv0007%Rv0147%Rv0148%Rv0149%Rv0821c%Rv0822c%Rv...
Module 155,Rv0039c%Rv1413%Rv2149c%Rv2150c%Rv2606c%Rv2637%...
Module 156,Rv0307c%Rv0597c%Rv0811c%Rv1084%Rv1239c%Rv1861%...
Module 157,Rv0040c%Rv0041%Rv0309%Rv0690c%Rv0692%Rv0693%Rv...
Module 158,Rv0188%Rv0270%Rv0426c%Rv1086%Rv1843c%Rv2182c%R...
Module 159,Rv0001%Rv0611c%Rv0739%Rv1155%Rv1167c%Rv1903%Rv...
Module 160,Rv0224c%Rv0494%Rv0998%Rv1374c%Rv1776c%Rv2771c%...


In [94]:
main(176, 176)

Module 176
['Rv1095', 'Rv1126c', 'Rv1127c', 'Rv2895c', 'Rv3688c', 'Rv3757c', 'Rv3758c', 'Rv3760', 'Rv3766', 'Rv1104']


In [95]:
main(200, 200)

Module 200
['Rv3504', 'Rv3505', 'Rv3515c', 'Rv3516', 'Rv3527', 'Rv3570c', 'Rv3573c', 'Rv3526', 'Rv3540c', 'Rv3543c', 'Rv3544c', 'Rv3545c', 'Rv3546', 'Rv3568c', 'Rv3574']


In [72]:
main(201, 250)

Module 201
['Rv0727c', 'Rv0728c', 'Rv1895', 'Rv2808', 'Rv3199c', 'Rv3329', 'Rv3782', 'Rv2880c', 'Rv3518c', 'Rv3781', 'Rv3783']
Module 202
['Rv0997', 'Rv1106c', 'Rv1222', 'Rv1284', 'Rv3229c', 'Rv3230c', 'Rv3235', 'Rv3847', 'Rv0426c', 'Rv3360', 'Rv3694c', 'Rv3695', 'Rv3802c']
Module 203
['Rv0568', 'Rv2751', 'Rv2752c', 'Rv2760c', 'Rv3056', 'Rv2750', 'Rv2757c', 'Rv2758c', 'Rv2766c']
Module 204
['Rv1371', 'Rv1372', 'Rv3322c', 'Rv3323c', 'Rv3324c', 'Rv3327', 'Rv0728c', 'Rv3473c']
Module 205
['Rv0795', 'Rv1370c', 'Rv1757c', 'Rv1764', 'Rv2168c', 'Rv2815c', 'Rv3786c', 'Rv1763', 'Rv2305', 'Rv2480c', 'Rv3184', 'Rv3186', 'Rv3325', 'Rv3381c', 'Rv3480c']
Module 206
['Rv0364', 'Rv0430', 'Rv0502', 'Rv0871', 'Rv1886c', 'Rv2144c', 'Rv2561', 'Rv2876', 'Rv3172c', 'Rv3260c', 'Rv3584', 'Rv3686c', 'Rv0740', 'Rv1860', 'Rv1887', 'Rv2376c']
Module 207
['Rv0090', 'Rv0371c', 'Rv0372c', 'Rv0374c', 'Rv0375c', 'Rv0573c', 'Rv0816c', 'Rv1672c', 'Rv1673c', 'Rv2328', 'Rv3175', 'Rv0150c', 'Rv0602c', 'Rv0904c', 'Rv1214c',

In [73]:
# response = requests.request("get", "http://networks.systemsbiology.net/mtb/biclusters/bicluster_0" + str(200), timeout = 20)
ModuleGene[200:250]
# droplist = []
# for i in range(250,251):
#     droplist.append("Module " + str(i))
# ModuleGene.drop(droplist,axis = 0,inplace = True)

# error 212 231 235 239

,using Gene
Module 201,Rv0727c%Rv0728c%Rv1895%Rv2808%Rv3199c%Rv3329%R...
Module 202,Rv0997%Rv1106c%Rv1222%Rv1284%Rv3229c%Rv3230c%R...
Module 203,Rv0568%Rv2751%Rv2752c%Rv2760c%Rv3056%Rv2750%Rv...
Module 204,Rv1371%Rv1372%Rv3322c%Rv3323c%Rv3324c%Rv3327%R...
Module 205,Rv0795%Rv1370c%Rv1757c%Rv1764%Rv2168c%Rv2815c%...
Module 206,Rv0364%Rv0430%Rv0502%Rv0871%Rv1886c%Rv2144c%Rv...
Module 207,Rv0090%Rv0371c%Rv0372c%Rv0374c%Rv0375c%Rv0573c...
Module 208,Rv1611%Rv1614%Rv2067c%Rv3438%Rv3718c%Rv2102%Rv...
Module 209,Rv0532%Rv0638%Rv1321%Rv1609%Rv1610%Rv1630%Rv19...
Module 210,Rv0021c%Rv0037c%Rv0217c%Rv0218%Rv0219%Rv0220%R...


In [96]:
main(212, 212)

Module 212
['Rv0879c', 'Rv1079', 'Rv2201', 'Rv2229c', 'Rv2563', 'Rv2567', 'Rv2992c', 'Rv2993c', 'Rv3233c', 'Rv3234c', 'Rv3575c', 'Rv3598c', 'Rv2199c', 'Rv2200c', 'Rv2230c', 'Rv2231c', 'Rv2994', 'Rv3576']


In [97]:
main(231, 231)

Module 231
['Rv0207c', 'Rv0208c', 'Rv0291', 'Rv1592c', 'Rv2243', 'Rv2244', 'Rv2245', 'Rv2246', 'Rv2247', 'Rv0206c', 'Rv0287', 'Rv0289', 'Rv0290', 'Rv0292', 'Rv1013', 'Rv3020c']


In [98]:
main(235, 235)

Module 235
['Rv0087', 'Rv0271c', 'Rv0373c', 'Rv0675', 'Rv0963c', 'Rv0987', 'Rv0988', 'Rv1503c', 'Rv1504c', 'Rv1740', 'Rv1741', 'Rv0084', 'Rv1739c', 'Rv1742', 'Rv1917c']


In [99]:
main(239, 239)

Module 239
['Rv0250c', 'Rv0681', 'Rv0730', 'Rv1882c', 'Rv1598c', 'Rv3160c', 'Rv3161c']


In [74]:
main(251, 300)

Module 251
['Rv0443', 'Rv0558', 'Rv1229c', 'Rv3376', 'Rv2334', 'Rv2335', 'Rv3396c', 'Rv3684']
Module 252
['Rv1410c', 'Rv1411c', 'Rv2152c', 'Rv2153c', 'Rv2197c', 'Rv2198c', 'Rv2537c', 'Rv2538c', 'Rv2539c', 'Rv2559c', 'Rv2753c', 'Rv2921c', 'Rv3305c', 'Rv3306c', 'Rv2157c', 'Rv2922c', 'Rv3593']
Module 253
['Rv0256c', 'Rv0565c', 'Rv0925c', 'Rv1515c', 'Rv1516c', 'Rv2519', 'Rv2958c', 'Rv3090', 'Rv3142c', 'Rv0731c', 'Rv2203']
Module 254
['Rv2237', 'Rv2296', 'Rv2473', 'Rv2472', 'Rv2654c', 'Rv2655c', 'Rv2656c', 'Rv2657c', 'Rv2819c', 'Rv2820c', 'Rv2821c', 'Rv2823c', 'Rv2824c']
Module 255
['Rv0247c', 'Rv0248c', 'Rv0632c', 'Rv2951c', 'Rv2952', 'Rv2971', 'Rv3464', 'Rv3465', 'Rv3846', 'Rv0164', 'Rv1099c', 'Rv1436', 'Rv1437', 'Rv1438', 'Rv2238c', 'Rv2239c', 'Rv2970c']
Module 256
['Rv1136', 'Rv1514c', 'Rv3015c', 'Rv3016', 'Rv3068c', 'Rv3069', 'Rv3070', 'Rv0735', 'Rv0736', 'Rv0985c', 'Rv0986', 'Rv3071']
Module 257
['Rv0501', 'Rv3144c', 'Rv3780', 'Rv0502', 'Rv1297', 'Rv1500', 'Rv3147', 'Rv3150', 'Rv3152'

In [75]:
ModuleGene[250:300]
#error 263 269 288 
#special 296

,using Gene
Module 251,Rv0443%Rv0558%Rv1229c%Rv3376%Rv2334%Rv2335%Rv3...
Module 252,Rv1410c%Rv1411c%Rv2152c%Rv2153c%Rv2197c%Rv2198...
Module 253,Rv0256c%Rv0565c%Rv0925c%Rv1515c%Rv1516c%Rv2519...
Module 254,Rv2237%Rv2296%Rv2473%Rv2472%Rv2654c%Rv2655c%Rv...
Module 255,Rv0247c%Rv0248c%Rv0632c%Rv2951c%Rv2952%Rv2971%...
Module 256,Rv1136%Rv1514c%Rv3015c%Rv3016%Rv3068c%Rv3069%R...
Module 257,Rv0501%Rv3144c%Rv3780%Rv0502%Rv1297%Rv1500%Rv3...
Module 258,Rv0277c%Rv0278c%Rv0438c%Rv0439c%Rv1759c%Rv2635...
Module 259,Rv1200%Rv1361c%Rv1803c%Rv1810%Rv2371%Rv2666%Rv...
Module 260,Rv0116c%Rv0117%Rv0893c%Rv0894%Rv1587c%Rv1588c%...


In [100]:
main(263, 263)
main(269, 269)
main(288, 288)

Module 263
['Rv0237', 'Rv0572c', 'Rv1175c', 'Rv2564', 'Rv2582', 'Rv2585c', 'Rv3012c', 'Rv3048c', 'Rv0361', 'Rv1176c', 'Rv2272', 'Rv2273', 'Rv2445c', 'Rv2561', 'Rv2563', 'Rv3738c']
Module 269
['Rv0168', 'Rv0177', 'Rv0178', 'Rv0249c', 'Rv0933', 'Rv0934', 'Rv0935', 'Rv0936', 'Rv1612', 'Rv1613', 'Rv2127', 'Rv2128', 'Rv0176', 'Rv0247c', 'Rv1614']
Module 288
['Rv0129c', 'Rv1157c', 'Rv1158c', 'Rv1433', 'Rv1477', 'Rv1478', 'Rv1770', 'Rv1771', 'Rv1987', 'Rv2151c', 'Rv2154c', 'Rv2155c', 'Rv2721c', 'Rv2927c', 'Rv2152c', 'Rv2153c']


In [76]:
main(301, 350)

Module 301
['Rv0201c', 'Rv0202c', 'Rv0203', 'Rv0583c', 'Rv0633c', 'Rv0951', 'Rv2450c', 'Rv2584c', 'Rv3274c', 'Rv3472', 'Rv3485c', 'Rv0547c', 'Rv0952', 'Rv2352c', 'Rv2725c', 'Rv3036c', 'Rv3436c', 'Rv3440c']
Module 302
['Rv2440c', 'Rv2809', 'Rv2906c', 'Rv2907c', 'Rv2908c', 'Rv2909c', 'Rv0464c', 'Rv0465c', 'Rv0466', 'Rv1239c', 'Rv1240', 'Rv2439c']
Module 303
['Rv0405', 'Rv1263', 'Rv1523', 'Rv0041', 'Rv0725c', 'Rv1191', 'Rv1193', 'Rv1629', 'Rv2481c', 'Rv3823c']
Module 304
['Rv0023', 'Rv0478', 'Rv2010', 'Rv2374c', 'Rv2596', 'Rv2704', 'Rv3356c', 'Rv3397c', 'Rv3441c', 'Rv0476', 'Rv0477', 'Rv1287', 'Rv2009', 'Rv2161c', 'Rv2595', 'Rv2665', 'Rv2694c']
Module 305
['Rv0534c', 'Rv0880', 'Rv0882', 'Rv2580c', 'Rv2581c', 'Rv2797c', 'Rv3482c', 'Rv3483c', 'Rv0551c', 'Rv0552', 'Rv0553', 'Rv0881', 'Rv2796c', 'Rv2798c', 'Rv2799', 'Rv2800']
Module 306
['Rv1081c', 'Rv1082', 'Rv1083', 'Rv1899c', 'Rv1900c', 'Rv1399c', 'Rv1400c', 'Rv1424c', 'Rv1901']
Module 307
['Rv2189c', 'Rv2544', 'Rv0239', 'Rv0240', 'Rv2543'

In [77]:
ModuleGene[300:350]
#error 314 316 326 338 346 347 349

,using Gene
Module 301,Rv0201c%Rv0202c%Rv0203%Rv0583c%Rv0633c%Rv0951%...
Module 302,Rv2440c%Rv2809%Rv2906c%Rv2907c%Rv2908c%Rv2909c...
Module 303,Rv0405%Rv1263%Rv1523%Rv0041%Rv0725c%Rv1191%Rv1...
Module 304,Rv0023%Rv0478%Rv2010%Rv2374c%Rv2596%Rv2704%Rv3...
Module 305,Rv0534c%Rv0880%Rv0882%Rv2580c%Rv2581c%Rv2797c%...
Module 306,Rv1081c%Rv1082%Rv1083%Rv1899c%Rv1900c%Rv1399c%...
Module 307,Rv2189c%Rv2544%Rv0239%Rv0240%Rv2543%Rv2549c%Rv...
Module 308,Rv1338%Rv2684%Rv2685%Rv3719%Rv3720%Rv1334%Rv13...
Module 309,Rv1430%Rv1876%Rv1981c%Rv2101%Rv2473
Module 310,Rv0883c%Rv2718c%Rv2738c%Rv3850%Rv2719c%Rv2739c...


In [101]:
main(314, 314)
main(316, 316)
main(326, 326)
main(338, 338)
main(346, 346)
main(347, 347)
main(349, 349)

Module 314
['Rv0382c', 'Rv0383c', 'Rv0660c', 'Rv1633', 'Rv1634', 'Rv2366c', 'Rv2367c', 'Rv2368c', 'Rv3010c', 'Rv3141', 'Rv0153c', 'Rv0154c', 'Rv0724', 'Rv0902c', 'Rv0903c', 'Rv2052c', 'Rv2053c']
Module 316
['Rv1435c', 'Rv1566c', 'Rv2145c', 'Rv2165c', 'Rv2166c', 'Rv2525c', 'Rv2986c', 'Rv3246c', 'Rv3258c', 'Rv3413c', 'Rv3414c', 'Rv3587c', 'Rv0430', 'Rv0660c', 'Rv0883c', 'Rv1497', 'Rv2927c', 'Rv3244c', 'Rv3245c', 'Rv3260c', 'Rv3415c']
Module 326
['Rv0975c', 'Rv0261c', 'Rv0971c', 'Rv0974c', 'Rv0976c', 'Rv0977', 'Rv1575', 'Rv1868', 'Rv1888c', 'Rv1999c', 'Rv2037c', 'Rv2040c', 'Rv2041c', 'Rv2042c', 'Rv2043c', 'Rv2285']
Module 338
['Rv2742c', 'Rv2767c', 'Rv3170', 'Rv1180', 'Rv1181', 'Rv2339', 'Rv3191c', 'Rv3479']
Module 346
['Rv0331', 'Rv2016', 'Rv2017', 'Rv2269c', 'Rv2270', 'Rv2321c', 'Rv2323c', 'Rv2324', 'Rv2667', 'Rv1375', 'Rv1376', 'Rv2011c', 'Rv2012', 'Rv2668', 'Rv2865', 'Rv2866', 'Rv3743c']
Module 347
['Rv1805c', 'Rv1808', 'Rv1809', 'Rv2872', 'Rv3033', 'Rv3081', 'Rv1804c', 'Rv1810', 'Rv2

In [78]:
main(351, 400)

Module 351
['Rv1003', 'Rv3432c', 'Rv3813c', 'Rv3815c', 'Rv3817', 'Rv1914c', 'Rv1915', 'Rv1916', 'Rv2298', 'Rv2609c', 'Rv2610c', 'Rv2611c', 'Rv2613c', 'Rv2614c', 'Rv2895c', 'Rv3814c', 'Rv3816c']
Module 352
['Rv0657c', 'Rv2175c', 'Rv2178c', 'Rv2187', 'Rv2291', 'Rv2304c', 'Rv3765c', 'Rv1076', 'Rv2188c', 'Rv3764c']
Module 353
['Rv3242c', 'Rv2312', 'Rv2637', 'Rv3612c']
Module 354
['Rv0315', 'Rv0667', 'Rv0668', 'Rv0682', 'Rv0683', 'Rv0684', 'Rv0685', 'Rv0700', 'Rv0701', 'Rv0702', 'Rv0704', 'Rv0705', 'Rv1649']
Module 355
['Rv1004c', 'Rv2191', 'Rv2449c', 'Rv2456c', 'Rv2693c', 'Rv2852c', 'Rv2853', 'Rv3202c', 'Rv3579c', 'Rv0478', 'Rv2849c', 'Rv2850c', 'Rv3201c', 'Rv3629c', 'Rv3828c']
Module 356
['Rv2140c', 'Rv2945c', 'Rv3304', 'Rv3671c', 'Rv0428c', 'Rv0429c', 'Rv0503c', 'Rv0504c', 'Rv1823', 'Rv1824', 'Rv1825', 'Rv1826', 'Rv1830', 'Rv2113', 'Rv2141c', 'Rv3669']
Module 357
['Rv1209', 'Rv3382c', 'Rv3383c', 'Rv3893c', 'Rv3896c', 'Rv0392c', 'Rv1210', 'Rv1705c', 'Rv1706c', 'Rv1745c', 'Rv1948c', 'Rv204

In [79]:
ModuleGene[350:400]
#error 360 380 387

,using Gene
Module 351,Rv1003%Rv3432c%Rv3813c%Rv3815c%Rv3817%Rv1914c%...
Module 352,Rv0657c%Rv2175c%Rv2178c%Rv2187%Rv2291%Rv2304c%...
Module 353,Rv3242c%Rv2312%Rv2637%Rv3612c
Module 354,Rv0315%Rv0667%Rv0668%Rv0682%Rv0683%Rv0684%Rv06...
Module 355,Rv1004c%Rv2191%Rv2449c%Rv2456c%Rv2693c%Rv2852c...
Module 356,Rv2140c%Rv2945c%Rv3304%Rv3671c%Rv0428c%Rv0429c...
Module 357,Rv1209%Rv3382c%Rv3383c%Rv3893c%Rv3896c%Rv0392c...
Module 358,Rv0075%Rv0570%Rv0900%Rv1036c%Rv1619%Rv0964c%Rv...
Module 359,Rv0444c%Rv1341%Rv1456c%Rv1457c%Rv1458c%Rv2257c...
Module 360,skip


In [102]:
main(360, 360)
main(380, 380)
main(387, 387)

Module 360
['Rv0911', 'Rv3361c', 'Rv3494c', 'Rv3495c', 'Rv3496c', 'Rv3497c', 'Rv3499c', 'Rv3501c', 'Rv3631', 'Rv3362c', 'Rv3363c', 'Rv3365c', 'Rv3492c', 'Rv3493c', 'Rv3498c', 'Rv3500c', 'Rv3632']
Module 380
['Rv0114', 'Rv0115', 'Rv1682', 'Rv2044c', 'Rv2826c', 'Rv2832c', 'Rv0253', 'Rv0388c', 'Rv0488', 'Rv1129c', 'Rv1985c', 'Rv1986', 'Rv2038c', 'Rv2039c', 'Rv2818c', 'Rv2834c', 'Rv2835c']
Module 387
['Rv3644c', 'Rv1443c', 'Rv2787', 'Rv3771c']


In [80]:
main(401, 450)

Module 401
['Rv0747', 'Rv0916c', 'Rv2638', 'Rv3122', 'Rv3123', 'Rv3288c', 'Rv0516c', 'Rv0751c', 'Rv0753c', 'Rv1187', 'Rv1991c', 'Rv1992c', 'Rv1993c', 'Rv2660c', 'Rv2661c', 'Rv2663', 'Rv3286c', 'Rv3287c', 'Rv3334']
Module 402
[]
Module 403
['Rv1085c', 'Rv1205', 'Rv1206', 'Rv2387', 'Rv2672', 'Rv0438c', 'Rv0439c', 'Rv1204c', 'Rv1227c', 'Rv1228', 'Rv1244', 'Rv2673']
Module 404
['Rv0306', 'Rv0571c', 'Rv1373', 'Rv1490', 'Rv2028c', 'Rv2669', 'Rv3079c', 'Rv0570', 'Rv1039c', 'Rv1040c', 'Rv1557', 'Rv1997', 'Rv2003c', 'Rv2026c', 'Rv2029c', 'Rv2624c', 'Rv2625c', 'Rv2666', 'Rv3468c']
Module 405
['Rv2365c', 'Rv2848c', 'Rv2851c', 'Rv0523c', 'Rv0524', 'Rv0525', 'Rv0808', 'Rv1025', 'Rv1026', 'Rv1095', 'Rv2325c', 'Rv2366c', 'Rv2367c', 'Rv2368c', 'Rv2852c', 'Rv2869c', 'Rv2870c', 'Rv3255c', 'Rv3256c', 'Rv3257c']
Module 406
['Rv0454', 'Rv0571c', 'Rv0827c', 'Rv0840c', 'Rv2004c', 'Rv2025c', 'Rv2089c', 'Rv3226c', 'Rv3238c']
Module 407
['Rv0487', 'Rv0799c', 'Rv0800', 'Rv0801', 'Rv1233c', 'Rv2575', 'Rv0036c', '

In [81]:
ModuleGene[400:450]
#402 411 432

,using Gene
Module 401,Rv0747%Rv0916c%Rv2638%Rv3122%Rv3123%Rv3288c%Rv...
Module 402,skip
Module 403,Rv1085c%Rv1205%Rv1206%Rv2387%Rv2672%Rv0438c%Rv...
Module 404,Rv0306%Rv0571c%Rv1373%Rv1490%Rv2028c%Rv2669%Rv...
Module 405,Rv2365c%Rv2848c%Rv2851c%Rv0523c%Rv0524%Rv0525%...
Module 406,Rv0454%Rv0571c%Rv0827c%Rv0840c%Rv2004c%Rv2025c...
Module 407,Rv0487%Rv0799c%Rv0800%Rv0801%Rv1233c%Rv2575%Rv...
Module 408,Rv0314c%Rv3353c%Rv3689%Rv3692%Rv0208c%Rv0360c%...
Module 409,Rv0341%Rv0342%Rv0343%Rv1100%Rv1898%Rv3007c%Rv3008
Module 410,Rv0032%Rv0033%Rv0034%Rv0035%Rv1355c%Rv3481c%Rv...


In [103]:
main(402,402)
main(411,411)#已经解决
main(432,432)

Module 402
['Rv0047c', 'Rv0350', 'Rv0351', 'Rv0352', 'Rv0353', 'Rv0814c', 'Rv0815c', 'Rv0859', 'Rv0860', 'Rv1636', 'Rv3117', 'Rv3118', 'Rv3219', 'Rv0046c', 'Rv3801c']
error
Module 411
[]
Module 432
['Rv0796', 'Rv0797', 'Rv1756c', 'Rv2106', 'Rv2167c', 'Rv2279', 'Rv2353c', 'Rv2355', 'Rv2479c', 'Rv2649', 'Rv2814c', 'Rv3185', 'Rv3187', 'Rv3326', 'Rv3380c', 'Rv3475']


In [83]:
main(451, 500)

Module 451
['Rv0542c', 'Rv0557', 'Rv1870c', 'Rv1871c', 'Rv1872c', 'Rv2901c']
Module 452
['Rv1148c', 'Rv1964', 'Rv1966', 'Rv1969', 'Rv1971', 'Rv0893c', 'Rv0894', 'Rv1249c', 'Rv1250', 'Rv1965', 'Rv1967', 'Rv1968', 'Rv1970', 'Rv1973', 'Rv2087']
Module 453
['Rv1670', 'Rv2768c', 'Rv2769c', 'Rv2862c', 'Rv3654c', 'Rv1671', 'Rv3022c', 'Rv3167c', 'Rv3777']
Module 454
['Rv2572c', 'Rv3821', 'Rv0583c', 'Rv1545', 'Rv2923c']
Module 455
['Rv1615', 'Rv1616', 'Rv2556c', 'Rv2508c', 'Rv2509', 'Rv3103c', 'Rv3104c', 'Rv3264c', 'Rv3265c', 'Rv3266c', 'Rv3267', 'Rv3268']
Module 456
['Rv0402c', 'Rv0404', 'Rv0015c', 'Rv0016c', 'Rv0017c', 'Rv0403c', 'Rv0405', 'Rv0424c', 'Rv0425c', 'Rv1165', 'Rv1215c', 'Rv1607', 'Rv2751', 'Rv3209', 'Rv3437', 'Rv3438']
Module 457
['Rv0899', 'Rv0901', 'Rv2183c', 'Rv2185c', 'Rv0900', 'Rv2184c', 'Rv2187', 'Rv3556c', 'Rv3557c']
Module 458
['Rv2138', 'Rv2785c', 'Rv3205c', 'Rv3285', 'Rv0049', 'Rv0462', 'Rv0463', 'Rv0558', 'Rv1294', 'Rv2074', 'Rv2241', 'Rv2309c', 'Rv2605c', 'Rv3050c', 'R

In [84]:
ModuleGene[450:500]
#461 491

,using Gene
Module 451,Rv0542c%Rv0557%Rv1870c%Rv1871c%Rv1872c%Rv2901c
Module 452,Rv1148c%Rv1964%Rv1966%Rv1969%Rv1971%Rv0893c%Rv...
Module 453,Rv1670%Rv2768c%Rv2769c%Rv2862c%Rv3654c%Rv1671%...
Module 454,Rv2572c%Rv3821%Rv0583c%Rv1545%Rv2923c
Module 455,Rv1615%Rv1616%Rv2556c%Rv2508c%Rv2509%Rv3103c%R...
Module 456,Rv0402c%Rv0404%Rv0015c%Rv0016c%Rv0017c%Rv0403c...
Module 457,Rv0899%Rv0901%Rv2183c%Rv2185c%Rv0900%Rv2184c%R...
Module 458,Rv2138%Rv2785c%Rv3205c%Rv3285%Rv0049%Rv0462%Rv...
Module 459,Rv2424c%Rv2458%Rv2722
Module 460,Rv0615%Rv2553c%Rv1005c%Rv1539%Rv1540%Rv1599%Rv...


In [105]:
main(461, 461)#已经解决
main(491, 491)

error
Module 461
[]
Module 491
['Rv2190c', 'Rv2700', 'Rv0021c', 'Rv0321', 'Rv0799c', 'Rv0950c', 'Rv1233c', 'Rv1589', 'Rv1590', 'Rv1591', 'Rv2907c', 'Rv2908c', 'Rv2909c', 'Rv3031', 'Rv3032', 'Rv3464', 'Rv3465']


In [106]:
main(461, 461)

Module 461
['Rv0658c', 'Rv1802', 'Rv2514c', 'Rv0124', 'Rv0341', 'Rv0342', 'Rv0343', 'Rv0698', 'Rv0746', 'Rv0833', 'Rv1087', 'Rv1419', 'Rv2249c', 'Rv2250c', 'Rv2515c', 'Rv2639c']


In [85]:
main(501, 550)

Module 501
['Rv0248c', 'Rv0249c', 'Rv0638', 'Rv1006', 'Rv2227', 'Rv2441c', 'Rv2443', 'Rv2519', 'Rv2520c', 'Rv2584c', 'Rv2904c', 'Rv2960c', 'Rv3136', 'Rv3404c', 'Rv3472', 'Rv3524', 'Rv3628']
Module 502
['Rv2378c', 'Rv2379c', 'Rv2380c', 'Rv2381c', 'Rv2382c', 'Rv2383c', 'Rv2384', 'Rv2386c', 'Rv0450c', 'Rv1342c', 'Rv1343c', 'Rv1344', 'Rv1345', 'Rv1346', 'Rv1347c', 'Rv1348', 'Rv1349', 'Rv1845c', 'Rv1846c', 'Rv1847']
Module 503
['Rv1827', 'Rv1828', 'Rv1984c', 'Rv3547', 'Rv1192', 'Rv1435c', 'Rv1780', 'Rv2147c', 'Rv2150c', 'Rv2986c', 'Rv2991', 'Rv3812', 'Rv3847', 'Rv3852']
Module 504
['Rv2447c', 'Rv3647c', 'Rv0201c', 'Rv0202c', 'Rv0203', 'Rv1209', 'Rv2146c', 'Rv2446c', 'Rv2448c', 'Rv2449c', 'Rv2738c', 'Rv2740', 'Rv2802c', 'Rv2862c', 'Rv3482c', 'Rv3483c', 'Rv3554', 'Rv3579c', 'Rv3580c']
Module 505
['Rv0008c', 'Rv1949c', 'Rv1953', 'Rv0009', 'Rv1081c', 'Rv1950c', 'Rv1951c', 'Rv1952', 'Rv2020c', 'Rv2021c', 'Rv2022c', 'Rv2023c', 'Rv3911', 'Rv3912']
Module 506
['Rv2236c', 'Rv2261c', 'Rv2262c', 'Rv22

In [86]:
ModuleGene[500:550]
#508 518 525 529 542

,using Gene
Module 501,Rv0248c%Rv0249c%Rv0638%Rv1006%Rv2227%Rv2441c%R...
Module 502,Rv2378c%Rv2379c%Rv2380c%Rv2381c%Rv2382c%Rv2383...
Module 503,Rv1827%Rv1828%Rv1984c%Rv3547%Rv1192%Rv1435c%Rv...
Module 504,Rv2447c%Rv3647c%Rv0201c%Rv0202c%Rv0203%Rv1209%...
Module 505,Rv0008c%Rv1949c%Rv1953%Rv0009%Rv1081c%Rv1950c%...
Module 506,Rv2236c%Rv2261c%Rv2262c%Rv2263%Rv1302%Rv1848%R...
Module 507,Rv0895%Rv0943c%Rv1509%Rv0095c%Rv0096%Rv0114%Rv...
Module 508,skip
Module 509,Rv0089%Rv0480c%Rv0626%Rv0627%Rv0818%Rv1160%Rv2...
Module 510,Rv0418%Rv0419%Rv0495c%Rv0496%Rv0565c%Rv1118c%R...


In [107]:
main(508, 508)
main(518, 518)
main(525, 525)
main(529, 529)
main(542, 542)
#508 有问题

Module 508
['Rv1070c', 'Rv1071c', 'Rv0066c', 'Rv1069c', 'Rv1072', 'Rv1326c', 'Rv1327c', 'Rv1328', 'Rv2606c', 'Rv2607']
Module 518
['Rv1647', 'Rv1648', 'Rv1787', 'Rv1790', 'Rv2397c', 'Rv2398c', 'Rv2399c', 'Rv2400c', 'Rv3054c', 'Rv3218']
Module 525
['Rv1519', 'Rv1773c', 'Rv1774', 'Rv1775', 'Rv2121c', 'Rv2122c', 'Rv2123', 'Rv2378c', 'Rv2379c', 'Rv2380c', 'Rv2381c', 'Rv2382c', 'Rv2383c', 'Rv2386c', 'Rv3402c']
Module 529
['Rv2977c', 'Rv2978c', 'Rv0538', 'Rv0605', 'Rv0606', 'Rv0920c', 'Rv0921', 'Rv0922', 'Rv1615', 'Rv1616', 'Rv1638', 'Rv1712', 'Rv2088', 'Rv2790c', 'Rv2791c', 'Rv2792c', 'Rv2979c', 'Rv2980']
Module 542
['Rv2008c', 'Rv1977', 'Rv2010', 'Rv2829c', 'Rv2830c', 'Rv2841c', 'Rv2842c', 'Rv2877c', 'Rv2878c', 'Rv2919c', 'Rv2920c', 'Rv3288c', 'Rv3289c', 'Rv3343c', 'Rv3345c', 'Rv3581c', 'Rv3582c']


In [111]:
main(549, 549)

Module 549
['Rv1930c', 'Rv0269c', 'Rv1063c', 'Rv1213', 'Rv1225c', 'Rv1983', 'Rv2310', 'Rv2488c', 'Rv2997', 'Rv3126c', 'Rv3127', 'Rv3817']


In [88]:
main(551, 600)

Module 551
[]
Module 552
['Rv3355c', 'Rv3357', 'Rv3358', 'Rv3609c', 'Rv3610c', 'Rv2319c', 'Rv2322c', 'Rv2506', 'Rv2572c', 'Rv2573', 'Rv2779c', 'Rv2966c', 'Rv3356c', 'Rv3361c', 'Rv3364c', 'Rv3382c', 'Rv3383c', 'Rv3611']
Module 553
['Rv2232', 'Rv0989c', 'Rv1573', 'Rv1673c', 'Rv1949c', 'Rv1964', 'Rv1966', 'Rv1969', 'Rv1972', 'Rv2282c', 'Rv2283', 'Rv2308', 'Rv2578c', 'Rv3469c', 'Rv3470c', 'Rv3659c']
Module 554
['Rv3275c', 'Rv3276c', 'Rv3320c', 'Rv3321c', 'Rv3330', 'Rv0184', 'Rv0379', 'Rv0544c', 'Rv0910', 'Rv1170', 'Rv1632c', 'Rv2062c', 'Rv2080', 'Rv2418c', 'Rv2671', 'Rv2672', 'Rv3689', 'Rv3692']
Module 555
['Rv2886c', 'Rv3021c', 'Rv0779c', 'Rv0836c', 'Rv0837c', 'Rv0981', 'Rv1582c', 'Rv1620c', 'Rv1748', 'Rv1752', 'Rv1777', 'Rv2170', 'Rv2577', 'Rv3273', 'Rv3349c', 'Rv3427c', 'Rv3637', 'Rv3714c']
error
Module 556
[]
Module 557
['Rv0091', 'Rv0052', 'Rv0062', 'Rv0256c', 'Rv0319', 'Rv0518', 'Rv0619', 'Rv0620', 'Rv0622', 'Rv0737', 'Rv2872', 'Rv3575c']
Module 558
['Rv0022c', 'Rv0107c', 'Rv0109', '

In [89]:
ModuleGene[550:600]
#error 556 562 564 591 592

,using Gene
Module 551,skip
Module 552,Rv3355c%Rv3357%Rv3358%Rv3609c%Rv3610c%Rv2319c%...
Module 553,Rv2232%Rv0989c%Rv1573%Rv1673c%Rv1949c%Rv1964%R...
Module 554,Rv3275c%Rv3276c%Rv3320c%Rv3321c%Rv3330%Rv0184%...
Module 555,Rv2886c%Rv3021c%Rv0779c%Rv0836c%Rv0837c%Rv0981...
Module 556,skip
Module 557,Rv0091%Rv0052%Rv0062%Rv0256c%Rv0319%Rv0518%Rv0...
Module 558,Rv0022c%Rv0107c%Rv0109%Rv0136%Rv0277c%Rv0278c%...
Module 559,Rv1037c%Rv1197%Rv1791%Rv2346c%Rv2347c%Rv3620c%...
Module 560,Rv2482c%Rv2483c%Rv2484c%Rv0186%Rv0188%Rv0232%R...


In [108]:
#处理所有爬的时候出问题的数据
main(556, 556)
main(562, 562)
main(564, 564)
main(591, 591)
main(592, 592)

Module 556
['Rv2068c', 'Rv2069', 'Rv2225', 'Rv2301', 'Rv2861c', 'Rv3142c', 'Rv3143', 'Rv3600c', 'Rv3601c', 'Rv3602c', 'Rv3603c', 'Rv3779']
Module 562
['Rv2096c', 'Rv2097c', 'Rv2133c', 'Rv2134c', 'Rv2135c', 'Rv3605c', 'Rv3606c', 'Rv3607c', 'Rv3608c', 'Rv3609c', 'Rv3610c', 'Rv3625c', 'Rv3626c', 'Rv3627c']
Module 564
['Rv0199', 'Rv1155', 'Rv1408', 'Rv1409', 'Rv2137c', 'Rv2138', 'Rv2197c', 'Rv2198c', 'Rv2342', 'Rv2405', 'Rv2579', 'Rv2718c']
Module 591
['Rv0139', 'Rv0297', 'Rv0499', 'Rv0729', 'Rv0845', 'Rv0854', 'Rv0855', 'Rv1259', 'Rv1260', 'Rv1459c', 'Rv1866', 'Rv1947', 'Rv2363', 'Rv2695', 'Rv2778c', 'Rv3021c', 'Rv3026c', 'Rv3027c', 'Rv3560c']
Module 592
['Rv2223c', 'Rv2224c', 'Rv2392', 'Rv2984', 'Rv3685c', 'Rv3753c']


In [112]:
main(551, 551)

Module 551
[]


In [ ]:
#最终296 318 551 565~573存在空缺

In [119]:
ModuleGene.head()

,using Gene
Module 1,Rv0830%Rv2377c%Rv2963%Rv2975c%Rv3848
Module 2,Rv0472c%Rv1871c%Rv3005c%Rv3006%Rv3037c%Rv3781%...
Module 3,Rv0395%Rv0396%Rv0689c%Rv0793%Rv0843%Rv0915c%Rv...
Module 4,Rv1408%Rv1409%Rv1923%Rv2305%Rv3403c%Rv3589%Rv3768
Module 5,Rv0809%Rv1101c%Rv1192%Rv1193%Rv1627c%Rv1628c%R...


In [117]:
LabGene.to_excel("LabGene1.0.xlsx", sheet_name="LabGene")

In [118]:
ModuleGene.to_excel("ModuleGene1.0.xlsx", sheet_name="ModuleGene")